<a href="https://colab.research.google.com/github/shliakhtas/ASCM/blob/main/night_light_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install earthengine-api geehydro geemap

     |████████████████████████████████| 389kB 7.4MB/s 
     |████████████████████████████████| 1.1MB 23.9MB/s 
     |████████████████████████████████| 1.9MB 32.4MB/s 
     |████████████████████████████████| 5.1MB 38.8MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
     |████████████████████████████████| 4.9MB 48.1MB/s 
     |████████████████████████████████| 225kB 50.0MB/s 
     |████████████████████████████████| 204kB 43.8MB/s 
     |████████████████████████████████| 552kB 42.9MB/s 
     |████████████████████████████████| 112kB 48.7MB/s 
     |████████████████████████████████| 122kB 44.4MB/s 
  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10114 sha256=d6a47dce2f3b6de7e285744a9030f3b6fbe4e4dde431f93568c94d1b05ae362a
  Stored in directory: /root/.cache/pip/wheels/c5/07/67/5fa6e7271b46bbe0acafdc7105bbee27a39ab7132d251d822d
  Created wheel for ipynb-py-convert: filename=ipynb_py_convert-0.4

In [5]:
!earthengine authenticate

from google.colab import drive
drive.mount('/content/drive')

# Import the Earth Engine API and initialize it.
import ee
ee.Initialize()

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=jy53RkeyfiUBJ2JMTblaOIDCLsvsk6ag2rmwDjZDgQ4&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AfDhmrgvFvNwHpv4onibLYeVY3jvoZMVErQBBVjqp2QMdD-6Ezct-V9Jszw

Successfully saved authorization token

In [11]:
import geehydro
import folium

# Load a Landsat 8 ImageCollection for a single path-row.
table = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', 'Ukraine'))
nighttimeLights = ee.ImageCollection('NOAA/DMSP-OLS/CALIBRATED_LIGHTS_V4').filterDate('2010-01-01', '2019-12-31').filterBounds(ee.Geometry.Rectangle(49.9398,  49.7572, 23.8752, 24.2065))
nighttimeLights = nighttimeLights.select('avg_vis')
image = ee.Image(nighttimeLights.median()).clip(table);

Map = folium.Map(location=[49.3022,32.1675], zoom_start=6)# To see a google satellite view as a basemap
Map.setOptions('HYBRID')

nighttimeLightsVis = {
  min: 3.0,
  max: 60.0,
};

Map.addLayer(image, nighttimeLightsVis, 'Night Ligth')
# Map.addLayer(table, {}, 'admin')

training = image.sample(**{
    'region': table,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})


n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)
# Cluster the input using the trained clusterer.
result = image.cluster(clusterer)

legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
legend_colors = ['#F0F8FF', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']

# Reclassify the map
result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

Map.addLayer(result, {'min': 1, 'max': 5, 'palette': legend_colors}, 'Labelled clusters')

Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

In [ ]:
# import branca

# colormap = branca.colormap.linear.YlOrRd_09.scale(0, 8500)
# colormap = colormap.to_step(index=[0, 1000, 3000, 5000, 8500])
# colormap.caption = 'Incidents of Crime in Victoria (year ending June 2018)'
# # colormap.colors=legend_colors
# colormap.add_to(Map)

# Map

In [12]:
import folium
from ipyleaflet import *
from bqplot import pyplot as plt


Map = folium.Map(location=[49.3022,32.1675], zoom_start=7)
Map.default_style = {'cursor': 'crosshair'}

# Compute the trend of nighttime lights from DMSP.
# Add a band containing image date as years since 1990.
def createTimeBand(img):
  year = img.date().difference(ee.Date('1991-01-01'), 'year')
  return ee.Image(year).float().addBands(img)

NTL = ee.ImageCollection('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS') \
    .select('stable_lights')

# Fit a linear trend to the nighttime lights collection.
collection = NTL.map(createTimeBand)
fit = collection.reduce(ee.Reducer.linearFit())

image = NTL.toBands()

figure = plt.figure(1, title='Nighttime Light Trend', layout={'max_height': '250px', 'max_width': '400px'})
count = collection.size().getInfo()
start_year = 1992
end_year = 2013
x = range(1, count+1)

coordinates = []
markers = []
# marker_cluster = MarkerCluster(name="Marker Cluster")
# Map.addLayer(marker_cluster)

def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click':
        coordinates.append(latlon)
        markers.append(Marker(location=latlon))
        marker_cluster.markers = markers
        xy = ee.Geometry.Point(latlon[::-1])
        y = image.sample(xy, 500).first().toDictionary().values().getInfo()
        plt.clear()
        plt.plot(x, y)
#         plt.xticks(range(start_year, end_year, 5))
# Map.on_interaction(handle_interaction)

# Display a single image
Map.addLayer(ee.Image(collection.select('stable_lights').first()), {'min': 0, 'max': 63}, 'First image')

# Display trend in red/blue, brightness in green.
Map.setCenter(30, 45, 4)
Map.addLayer(fit,
         {'min': 0, 'max': [0.18, 20, -0.18], 'bands': ['scale', 'offset', 'scale']},
         'stable lights trend')

# fig_control = WidgetControl(widget=figure, position='bottomright')
# Map.add_control(fig_control)

Map